In [1]:
import requests
import pandas as pd

In [2]:
sdf = pd.read_csv('../Data/AU.txt', sep="\t", 
                 names=['Country','Postcode','Suburb','StName','State','City','B1','B22','B3','Lat','Long','Other'])

In [3]:
property_types = [
    '2bdHouse','3bdHouse','4bdHouse','1bdUnit','2bdUnit','3bdUnit'
]


In [16]:
df = sdf.copy()

In [ ]:
df['Suburb'] = df['Suburb'].astype(str).str.strip().str.replace(' ','-')
df['Postcode'] = df['Postcode'].astype(str).str.zfill(4)
success_record = {}

adf = df.copy()#[df['State']=='VIC']

with open('../Outputs/success_record.txt','w') as outfile:

    for sub,st,pc in list(zip(adf['Suburb'].astype(str).str.strip().str.replace(' ','-'),
                              adf['State'].astype(str),
                              adf['Postcode'].astype(str).str.zfill(4))):
        r = requests.get('http://www.domain.com.au/suburb-profile/'+"-".join([sub,st,pc]))
        try:
            r.raise_for_status()
            df.loc[
                (df['Suburb'].astype(str)==sub)&
                (df['State'].astype(str)==st)&
                (df['Postcode'].astype(str)==pc),
                'Rental Percentage'
            ] = r.text.split('data-label="renting">')[1].split("<")[0]
            df.loc[
                (df['Suburb'].astype(str)==sub)&
                (df['State'].astype(str)==st)&
                (df['Postcode'].astype(str)==pc),
                'Purchasing Percentage'
            ] = r.text.split('data-label="purchasing">')[1].split("<")[0]
            try:
                for property_type in property_types:
                    nullcount = 0
                    a = r.text.split(property_type)[1].split("mediansoldprice\":")[1].split(",")[0]
                    if a.lower() == "null":
                        nullcount += 1
                    df.loc[
                        (df['Suburb'].astype(str)==sub)&
                        (df['State'].astype(str)==st)&
                        (df['Postcode'].astype(str)==pc),
                        property_type
                    ] = a
                print('-'.join([sub,st,pc])+' was successful! Inserted to table. There were '+
                      str(nullcount)+' null prices')
                outfile.write('-'.join([sub,st,pc,'successful','yes_prices']))
            except:
                print("Couldn't find property info for "+'-'.join([sub,st,pc]))
                outfile.write('-'.join([sub,st,pc,'successful','no_prices']))
                None
            try:
                df.loc[
                        (df['Suburb'].astype(str)==sub)&
                        (df['State'].astype(str)==st)&
                        (df['Postcode'].astype(str)==pc),
                        'historical_prices'
                    ] = r.text.split('data: [')[1].split(']')[0]
            except:
                print('No historical prices for '+'-'.join([sub,st,pc]))
                None
            try:
                df.loc[
                            (df['Suburb'].astype(str)==sub)&
                            (df['State'].astype(str)==st)&
                            (df['Postcode'].astype(str)==pc),
                            'unit_historical_prices'
                        ] = r.text.split('data: [')[2].split(']')[0]
            except:
                print(sub+"-"+st+"-"+pc,"units hp failed")
        except:
            print('-'.join([sub,st,pc])+' threw an error')
            outfile.write('-'.join([sub,st,pc,'failed','completely']))
            continue

Australian-National-University-ACT-0200 threw an error
Barton-ACT-0221 threw an error
Jervis-Bay-ACT-2540 threw an error
Wreck-Bay-ACT-2540 threw an error
Hmas-Creswell-ACT-2540 threw an error
Duntroon-ACT-2600 threw an error
Deakin-West-ACT-2600 threw an error
Harman-ACT-2600 threw an error
Capital-Hill-ACT-2600 threw an error
Barton-ACT-2600 was successful! Inserted to table. There were 1 null prices
Canberra-ACT-2600 was successful! Inserted to table. There were 1 null prices
Parliament-House-ACT-2600 threw an error
Russell-ACT-2600 threw an error
Yarralumla-ACT-2600 was successful! Inserted to table. There were 1 null prices
Deakin-ACT-2600 was successful! Inserted to table. There were 1 null prices
Parkes-ACT-2600 was successful! Inserted to table. There were 1 null prices
Hmas-Harman-ACT-2600 threw an error
Canberra-ACT-2601 threw an error
City-ACT-2601 was successful! Inserted to table. There were 1 null prices
Acton-ACT-2601 was successful! Inserted to table. There were 1 null 

In [26]:
df[pd.notnull(df['Rental Percentage'])].drop(['B1','B22','B3'],axis=1
                                            ).to_csv('../Outputs/complete_table_2.csv', index=False)